# Approach A

In [1]:
identifier = 'bD_Z2_S2'
identifier_specify = identifier[-5:]
tex = f"{identifier}/output/calibro_report.tex"
_idfPath = f"../eplus_diagnosis/idf-Oct/G2/auto-cal/bD_alpha_{identifier_specify}.idf"

output_name = f'../eplus_diagnosis/idf-Oct/G2/POST-cal/{identifier}'
output_name = f'{identifier}'
import sys
from pathlib import Path
from _approach_a import *
from _approach_b import *
here = Path.cwd()
src_dir = next(p for p in [here, *here.parents] if (p / "src").exists()) / "src"
sys.path.insert(0, str(src_dir))
from ReIDF import ReIDF1

In [2]:
def generate_office_string(pattern, ending=';'):
    result = []
    for value, count in pattern:
        result.extend([value] * count)
    return ', '.join(result) + ending

# Defaults (model's baseline values if no calibrated value exists)
d1_htgsp_office_default = 18
d2_htgsp_office_st_default = 15

# Placeholder schedules (raw form prior to replacement)
d1_htg_updated_weekday = generate_office_string([
    ('15', 8), ('{{d1_htgsp_office}}', 11), ('15', 5)
])

d2_htg_updated_weekend = generate_office_string([('{{d2_htgsp_office_st}}', 24)])

d2_htg_updated_weekday = generate_office_string([
    ('{{d2_htgsp_office_st}}', 8), ('{{d1_htgsp_office}}', 11), ('{{d2_htgsp_office_st}}', 5)
])

# Original user-defined lists (must preserve format)
ReUnit_HVAC = [
    ('MaterialExt_W', 3, '{{a1_u_value_external_walls}}'),
    ('MaterialExt_Win', 2, '{{a3_u_value_external_windows}}'),
    ('MaterialExt_Win 1', 2, '{{a3_u_value_external_windows}}'),
    ('MaterialExt_Win 2', 2, '{{a3_u_value_external_windows}}'),
    ('MaterialExt_Win', 3, '{{a4_g_value}}'),
    ('MaterialExt_Win 1', 3, '{{a4_g_value}}'),
    ('MaterialExt_Win 2', 3, '{{a4_g_value}}'),
    ('htg_sch_office_br__Weekday', 3, d1_htg_updated_weekday),
    ('htg_sch_office_br__Weekday', 3, d2_htg_updated_weekday),
    ('htg_sch_office_br__Sunday', 3, d2_htg_updated_weekend),
]

ReClass_HVAC = [
    ('ZoneVentilation:WindandStackOpenArea', 3, '{{e1_natural_ventilation_rate}}'),
    ('ZoneVentilation:WindandStackOpenArea', 7, '{{e1_natural_ventilation_rate}}'),
    ('ZoneInfiltration:DesignFlowRate', 7, '{{a5_infiltration_rate}}'),
    ('Fan:ConstantVolume', 3, '{{b1_ahu_fan_efficiency}}'),
    ('AirLoopHVAC', 11, '{{b10_airloophvac}};'),
    ('SetpointManager:OutdoorAirReset', 3, '{{b12_sat}}'),
]


In [3]:
df, param_map, ReUnit_CalibroMID, ReClass_CalibroMID = process_all(
    tex,
    ReUnit_HVAC,
    ReClass_HVAC,
    drop_missing=True 
)
ReClass_CalibroMID, ReUnit_CalibroMID

([('ZoneVentilation:WindandStackOpenArea', 3, '1.223'),
  ('ZoneVentilation:WindandStackOpenArea', 7, '1.223'),
  ('ZoneInfiltration:DesignFlowRate', 7, '0.872'),
  ('SetpointManager:OutdoorAirReset', 3, '23.360')],
 [('htg_sch_office_br__Weekday',
   3,
   '15, 15, 15, 15, 15, 15, 15, 15, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 15, 15, 15, 15, 15;'),
  ('htg_sch_office_br__Weekday',
   3,
   '17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 23.779, 17.775, 17.775, 17.775, 17.775, 17.775;'),
  ('htg_sch_office_br__Sunday',
   3,
   '17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775, 17.775;')])

In [4]:
_idfPath = Path(_idfPath)
updated = ReIDF1(oriPath=_idfPath, 
                 ReClass=ReClass_CalibroMID, 
                 ReUnit=ReUnit_CalibroMID, 
                 cusPath=_idfPath.parent / ".." / "post-cal" / ( 'POST' + _idfPath.stem + _idfPath.suffix))

ReIDF: → /Users/rui.bo/Desktop/Working/1-phd_mainworks/Y3/eplus_diagnosis/idf-Oct/G2/post-cal/POSTbD_alpha_Z2_S2.idf
ReUnit: [{'htg_sch_office_br__Weekday': 1}, {'htg_sch_office_br__Weekday': 1}, {'htg_sch_office_br__Sunday': 1}]
ReClass: [{'ZoneVentilation:WindandStackOpenArea': 15}, {'ZoneVentilation:WindandStackOpenArea': 15}, {'ZoneInfiltration:DesignFlowRate': 14}, {'SetpointManager:OutdoorAirReset': 1}]


# Approach B

In [5]:
df, param_map, ReUnit_PLACEHOLDER, ReClass_PLACEHOLDER = process_all(
    tex,
    ReUnit_HVAC,
    ReClass_HVAC,
    drop_missing=True,
    overwrite_with_derived=False  # Keep original placeholders but drop unresolved ones
)

ReUnit_PLACEHOLDER, ReClass_PLACEHOLDER


([('htg_sch_office_br__Weekday',
   3,
   '15, 15, 15, 15, 15, 15, 15, 15, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, 15, 15, 15, 15, 15;'),
  ('htg_sch_office_br__Weekday',
   3,
   '{{d2_htgsp_office_st}}, {{d2_htgsp_office_st}}, {{d2_htgsp_office_st}}, {{d2_htgsp_office_st}}, {{d2_htgsp_office_st}}, {{d2_htgsp_office_st}}, {{d2_htgsp_office_st}}, {{d2_htgsp_office_st}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d1_htgsp_office}}, {{d2_htgsp_office_st}}, {{d2_htgsp_office_st}}, {{d2_htgsp_office_st}}, {{d2_htgsp_office_st}}, {{d2_htgsp_office_st}};'),
  ('htg_sch_office_br__Sunday',
   3,
   '{{d2_htgsp_office_st}}, {{d2_htgsp_offic

In [6]:
updated = Path(updated)
_updated = ReIDF1(oriPath=updated, 
                 ReClass=ReClass_PLACEHOLDER, 
                 ReUnit=ReUnit_PLACEHOLDER, 
                 cusPath=_idfPath.parent / ".." / "post-cal" / ( 'UQPOST' + _idfPath.stem + _idfPath.suffix))

ReIDF: → /Users/rui.bo/Desktop/Working/1-phd_mainworks/Y3/eplus_diagnosis/idf-Oct/G2/post-cal/UQPOSTbD_alpha_Z2_S2.idf
ReUnit: [{'htg_sch_office_br__Weekday': 1}, {'htg_sch_office_br__Weekday': 1}, {'htg_sch_office_br__Sunday': 1}]
ReClass: [{'ZoneVentilation:WindandStackOpenArea': 15}, {'ZoneVentilation:WindandStackOpenArea': 15}, {'ZoneInfiltration:DesignFlowRate': 14}, {'SetpointManager:OutdoorAirReset': 1}]


In [7]:
user_bounds = {
    'b10_airloophvac': (None, 1),            # upper bound manually set to 1
    'a5_infiltration_rate': (0, None),        # force lower bound at 0
}
infer_distributions_with_trunc(df, user_bounds= user_bounds)

,Parameter,Distribution,Mean,Mu_log,Sigma_log,CV,Lower,Upper
0,d1_htgsp_office,Lognormal,23.779,3.167735,0.046213,0.046238,21.624,23.977
1,d2_htgsp_office_st,Lognormal,17.775,2.876232,0.055871,0.055914,15.827,17.980
2,e1_natural_ventilation_rate,Lognormal,1.223,0.192365,0.133730,0.134330,0.901,1.346
3,a5_infiltration_rate,Lognormal,0.872,-0.149111,0.155855,0.156806,0.604,0.968
4,b12_sat,Lognormal,23.360,3.141552,0.137642,0.138297,17.028,27.049


# Archive

In [ ]:
print(dist_df.columns.tolist())
print(dist_df.head())


['Parameter', 'Distribution', 'Mean', 'Mu_log', 'Sigma_log', 'CV', 'Lower', 'Upper']
                     Parameter Distribution    Mean    Mu_log  Sigma_log  \
0              d1_htgsp_office    Lognormal  23.779  3.167735   0.046213   
1           d2_htgsp_office_st    Lognormal  17.775  2.876232   0.055871   
2  e1_natural_ventilation_rate    Lognormal   1.223  0.192365   0.133730   
3         a5_infiltration_rate    Lognormal   0.872 -0.149111   0.155855   
4                      b12_sat    Lognormal  23.360  3.141552   0.137642   

         CV   Lower   Upper  
0  0.046238  21.624  23.977  
1  0.055914  15.827  17.980  
2  0.134330   0.901   1.346  
3  0.156806   0.604   0.968  
4  0.138297  17.028  27.049  
